In [1]:
import torch
from torch import nn

import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt

torch.__version__

'2.1.2+cpu'

### 0.Importing PyTorch

In [2]:
###### setting up device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

### 1.Get data

In [4]:
import requests
import zipfile
from pathlib import Path


data_path = Path("data/")
image_path = data_path / "archive"

if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    image_path.mkdir(parents=True, exist_ok=True)

    with zipfile.ZipFile(data_path/ "archive.zip", "r") as zip_ref:
        print("unizipping...")
        zip_ref.extractall(image_path)

unizipping...


In [3]:
train_data = datasets.FashionMNIST(
    root= 'data', # where to download data to?
    train= True,  # training data
    download= True, # download data if it doesnt exist on disk
    transform= ToTensor(), # images come as PIL format, we want to turn into Torch tensors
    target_transform=None # transform labels
)

test_data =  datasets.FashionMNIST(
    root= 'data',
    train= False,
    download= True,
    transform= ToTensor()
)

In [4]:
image, label= train_data[0]
image.shape

torch.Size([1, 28, 28])

In [5]:
class_names = train_data.classes
class_names

['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']

In [6]:
#Prepare DataLoader

from torch.utils.data import DataLoader

# Setup the batch size hyperparameter
Batch_size = 32

#Turn datasets into iterables(batches)
train_dataloader = DataLoader(train_data, batch_size=Batch_size, shuffle= True)

test_dataloader = DataLoader(test_data, batch_size=Batch_size, shuffle=False)

print(f"Data loaders:{train_dataloader, test_dataloader}")
print(f"length of train dataloader: {len(train_dataloader)} batches of {Batch_size}")
print(f"length of test dataloader: {len(test_dataloader)} batches of {Batch_size}")



Data loaders:(<torch.utils.data.dataloader.DataLoader object at 0x000001D96E1764C0>, <torch.utils.data.dataloader.DataLoader object at 0x000001D962D029D0>)
length of train dataloader: 1875 batches of 32
length of test dataloader: 313 batches of 32


In [7]:
#whats inside the training dataloader
train_features_batch, train_labels_batch = next(iter(train_dataloader))
train_features_batch.shape, train_labels_batch.shape

(torch.Size([32, 1, 28, 28]), torch.Size([32]))

### 2.Model Building

`nn.Flatten()` compress the dimensions of a tensor into a single vector

In [8]:
#Create a flatten layer
flatten_model = nn.Flatten()

#Get a single sample
x= train_features_batch[0]

#Flatten the sample 
output= flatten_model(x)

#print out what happened
print(f"shape before flattening: {x.shape} -> [color_channels, height, width]")
print(f"shape after flattening: {output.shape} -> [color_channels, height*width]")

shape before flattening: torch.Size([1, 28, 28]) -> [color_channels, height, width]
shape after flattening: torch.Size([1, 784]) -> [color_channels, height*width]


In [9]:
print(x)
print(output)

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,

we've turned our pixel data from height and width dimensions into one long feature vector

In [ ]:
from torch import nn
class FashionMNISTModelV0(nn.Module):
    def __init__(self, input_shape: int, hidden_units:int, output_shape:int):
        super().__init__()
        self.layer_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=input_shape, out_features=hidden_units),
            nn.Linear(in_features=hidden_units, out_features=output_shape)
        )

    def forward(self,x):
        return self.layer_stack(x)